This will be the Coursera's Capstone project

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Read the wiki page

In [7]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

Find the table rows

In [8]:
trs = soup.find("table").find("tbody").find_all("tr")

Read table data into the dataframe

In [9]:
row_data = []
for tr in trs:
    td = tr.find_all("td")
    row = [t.text for t in td]
    if len(row) == 3 and row[1] != "Not assigned":
        row[2] = row[2].rstrip()
        if row[2] == "Not assigned":
            row[2] = row[1]
        row_data.append(row)

df = pd.DataFrame(row_data, columns=["PostalCode", "Borough", "Neighborhood"])

Preview data

In [10]:
print(df.shape)

(211, 3)
